In [1]:
# pip install ollama

In [1]:
# model = 'llama3.2:3b-instruct-fp16'
# model = 'llama3-chatqa:latest'
# model = 'llama3-groq-tool-use'
model = 'qwen2.5:14b'

In [2]:
# ollama run llama3.2:3b-instruct-fp16 --keepalive 60m

### structured output - async

In [3]:
import asyncio

from pydantic import BaseModel

from ollama import AsyncClient


# Define the schema for the response
class FriendInfo(BaseModel):
  name: str
  age: int
  is_available: bool


class FriendList(BaseModel):
  friends: list[FriendInfo]


async def main():
  client = AsyncClient()
  response = await client.chat(
    model=model,
    messages=[{'role': 'user', 'content': 'I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format'}],
    format=FriendList.model_json_schema(),  # Use Pydantic to generate the schema
    options={'temperature': 0},  # Make responses more deterministic
  )

  # Use Pydantic to validate the response
  friends_response = FriendList.model_validate_json(response.message.content)

  print(friends_response)
  return friends_response


# if __name__ == '__main__':
#   asyncio.run(main())

In [4]:
fr = await main()

friends=[FriendInfo(name='Ollama', age=22, is_available=False), FriendInfo(name='Alonso', age=23, is_available=True)]


In [5]:
fr.friends

[FriendInfo(name='Ollama', age=22, is_available=False),
 FriendInfo(name='Alonso', age=23, is_available=True)]

### normal chat -async

In [19]:
import asyncio

from ollama import AsyncClient


async def main():
  messages = [
    {
      'role': 'user',
      'content': 'Why is the sky blue?',
    },
  ]

  client = AsyncClient()
  response = await client.chat(model, messages=messages)
  print(response['message']['content'])

await main()

The sky appears blue because of a phenomenon called Rayleigh scattering, which is the scattering of light by small particles or molecules in the atmosphere. The shorter, blue wavelengths are scattered more than the longer, red wavelengths, resulting in the blue color we see.

Here's how it works:

1. Sunlight enters Earth's atmosphere and is made up of all colors of the visible spectrum.
2. When this light encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2), it scatters.
3. The shorter wavelengths, like blue and violet, are scattered more than the longer wavelengths, such as red and orange.
4. This scattering effect gives the sky its blue color during the daytime.

The color can vary depending on atmospheric conditions, such as pollution, dust, or water vapor, but in general, the sky appears blue due to Rayleigh scattering.


### tool calling - async

In [16]:
# pip install duckduckgo-search

In [17]:
from duckduckgo_search import DDGS
DDGS().text('prompt programming',max_results=5)

[{'title': '[2102.07350] Prompt Programming for Large Language Models: Beyond the Few-Shot Paradigm',
  'href': 'https://arxiv.org/abs/2102.07350',
  'body': "Prevailing methods for mapping large generative language models to supervised tasks may fail to sufficiently probe models' novel capabilities. Using GPT-3 as a case study, we show that 0-shot prompts can significantly outperform few-shot prompts. We suggest that the function of few-shot examples in these cases is better described as locating an already learned task rather than meta-learning ..."},
 {'title': 'How to Become a Prompt Engineer: Duties, Skills, and Steps',
  'href': 'https://www.coursera.org/articles/how-to-become-a-prompt-engineer',
  'body': 'Writing skills ensure that you write prompts that are clear to the language model and natural to the user. Practice writing commands or questions using a conversational tone. Refine prompts in a "chat" to teach the AI how to produce a better output. You can change words and se

In [18]:
import inspect
import json

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

def sample_function(param_1, param_2, the_third_one: int, some_optional="John Doe"):
    """
    This is my docstring. Call this function when you want.
    """
    print("Hello, world")

schema =  function_to_schema(sample_function)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "sample_function",
    "description": "This is my docstring. Call this function when you want.",
    "parameters": {
      "type": "object",
      "properties": {
        "param_1": {
          "type": "string"
        },
        "param_2": {
          "type": "string"
        },
        "the_third_one": {
          "type": "integer"
        },
        "some_optional": {
          "type": "string"
        }
      },
      "required": [
        "param_1",
        "param_2",
        "the_third_one"
      ]
    }
  }
}


In [22]:
from typing import *

List[Dict]

typing.List[typing.Dict]

In [26]:
import asyncio
from typing import *
import ollama
from ollama import ChatResponse
from pydantic import BaseModel, Field


class ResearchResult(BaseModel):
    research_title: str = Field(description='This is a top level Markdown heading that covers the topic of the search query #')
    research_main: str = Field(description='This is a main section that provides detailed search result description for the query and research')





def duck_search(query:str) -> List[Dict]:
    """
    Search internet for given query in detail.
    
    Args:
    query (str): detailed search query
    
    
    Returns:
    List[Dict]: search results.
    """
    search_result = DDGS().text(query,max_results=5)
    
    return search_result





# Tools can still be manually defined and passed into chat
duck_search_tool = function_to_schema(duck_search)
system_messages = [{'role': 'system', 'content': 'Behave as general ai assistant, call tools if ask by user'}]
messages = system_messages+[{'role': 'user', 'content': 'do that tool for tell me about todays top news 1feb 2025'}]
print('Prompt:', messages[1]['content'])

available_functions = {
  'duck_search': duck_search,
}


async def main():
  client = ollama.AsyncClient()

  response: ChatResponse = await client.chat(
    model,
    messages=messages,
    tools=[duck_search_tool],
  )

  print(response)
  if response.message.tool_calls:
    # There may be multiple tool calls in the response
    for tool in response.message.tool_calls:
      # Ensure the function is available, and then call it
      if function_to_call := available_functions.get(tool.function.name):
        print('Calling function:', tool.function.name)
        print('Arguments:', tool.function.arguments)
        output = function_to_call(**tool.function.arguments)
        print('Function output:', output)
      else:
        print('Function', tool.function.name, 'not found')

  # Only needed to chat with the model using the tool call results
  if response.message.tool_calls:
    # Add the function response to messages for the model to use
    messages.append(response.message)
    messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

    # Get final response from model with function outputs
    final_response = await client.chat(model, messages=messages,format=ResearchResult.model_json_schema(),)
    print('Final response:', final_response.message.content)

  else:
    print('No tool calls returned from model')


# if __name__ == '__main__':
#   try:
#     asyncio.run(main())
#   except KeyboardInterrupt:
#     print('\nGoodbye!')

await(main())

Prompt: do that tool for tell me about todays top news 1feb 2025
model='llama3-groq-tool-use' created_at='2025-02-01T15:39:30.306340439Z' done=True done_reason='stop' total_duration=430604684 load_duration=29348689 prompt_eval_count=207 prompt_eval_duration=25000000 eval_count=34 eval_duration=372000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='duck_search', arguments={'query': 'top news 1st February 2025'}))])
Calling function: duck_search
Arguments: {'query': 'top news 1st February 2025'}
Function output: [{'title': 'February 2025 News Archive - The Wall Street Journal', 'href': 'https://www.wsj.com/news/archive/2025/february', 'body': "WSJ's digital archive of news articles and top headlines from February 2025"}, {'title': 'February 1, 2025 - New York Post', 'href': 'https://nypost.com/2025/02/01/', 'body': 'Read the news from February 1, 2025 on the New York Post.'}, {'title': 'World News Live Today February 1, 2025: Sma

### chat with history

In [62]:
from ollama import chat

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
  {
    'role': 'assistant',
    'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight.",
  },
  {
    'role': 'user',
    'content': 'What is the weather in Tokyo?',
  },
  {
    'role': 'assistant',
    'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing. The city is known for its high-tech and vibrant culture, with many popular tourist attractions such as the Tokyo Tower, Senso-ji Temple, and the bustling Shibuya district.',
  },
]

while True:
  user_input = input('Chat with history: ').strip()
  if user_input.lower()=='q':
    break
    
  response = chat(
    model,
    messages=messages
    + [
      {'role': 'user', 'content': user_input},
    ],
  )

  # Add the response to the messages to maintain the history
  messages += [
    {'role': 'user', 'content': user_input},
    {'role': 'assistant', 'content': response.message.content},
  ]
  print(response.message.content + '\n')

Chat with history:  hi


Hello! How can I assist you today?



Chat with history:  q


## chat-stream

In [34]:
from ollama import chat

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
]

for part in chat(model, messages=messages, stream=True):
  print(part['message']['content'], end='', flush=True)

print()

The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it collides with molecules and small particles in the air. Sunlight is made up of different colors, each color corresponding to a different wavelength. Blue light waves are shorter and scatter more than other colors when they strike gas molecules (such as nitrogen and oxygen) in the atmosphere.

This scattered blue light enters our eyes from all directions, making the sky appear blue during clear days. Other colors also get scattered but not as much as blue because their wavelengths are longer. During sunrise or sunset, the sky appears red or orange because sunlight travels a longer path through the atmosphere, scattering away more of the shorter wavelengths and leaving behind mostly the longer ones.

It's worth noting that while our sky often looks blue, other planets with different atmospheres might show skies in other colors due to their unique atmospheric compositions.


In [20]:
from typing import *

In [21]:
def duck_search(query:str) -> List[Dict]:
    """
    Search internet for given query in detail.
    
    Args:
    query (str): detailed search query
    
    
    Returns:
    List[Dict]: search results.
    """
    search_result = DDGS().text(query,max_results=5)
    
    return search_result

In [22]:
tool_schemas = [function_to_schema(duck_search)]

In [23]:
tool_schemas

[{'type': 'function',
  'function': {'name': 'duck_search',
   'description': 'Search internet for given query in detail.\n    \n    Args:\n    query (str): detailed search query\n    \n    \n    Returns:\n    List[Dict]: search results.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string'}},
    'required': ['query']}}}]

In [46]:
from ollama import Client
ollama_client  = Client()
messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
]

for part in ollama_client.chat(model, messages=messages,tools=tool_schemas, stream=True):
  print(part['message'], flush=True)

role='assistant' content='' images=None tool_calls=[ToolCall(function=Function(name='duck_search', arguments={'query': 'Why is the sky blue?'}))]
role='assistant' content='' images=None tool_calls=None


In [14]:
# ollama_client??

In [45]:
part['message']

Message(role='assistant', content='', images=None, tool_calls=None)